In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from apple_simulator import simulate, _simulate, solution_to_df
from apple_regression import features_simple, features_drag, features_overfit, fit
import statsmodels.api as sm
from vessel_manoeuvring_models.extended_kalman_filter import extended_kalman_filter, time_steps_to_df, variance
from apple_simulator import step
from vessel_manoeuvring_models.substitute_dynamic_symbols import lambdify, run
from wPCC_pipeline.presentation import set_file_name, snapshot

In [ ]:
from string import ascii_uppercase as alc
alc_ = str(alc)

def label_reset():
    global alc_
    alc_ = str(alc)
    
def label(parameters, letter=None):
    
    if letter is None:
        global alc_
        letter = alc_[0]
        alc_ = str(alc_[1:])
        
    return f"{letter} (" + "".join(f"{key}: {np.round(value,4)}, " for key,value in parameters.items() if key!='m') + ")"

$$ a = \frac{F}{m} $$

$$ F=-m \cdot g + C_d \cdot v^2 + k \cdot x$$

$$ a=\frac{-m \cdot g}{m} = -g$$

$$v= \int_{0}^{T} a \,dt $$
$$x= \int_{0}^{T} v \,dt $$

In [ ]:
t = np.arange(0, 20, 0.25)
#t = np.arange(0, 20, 0.01)

parameters: dict = {
        "g": 9.81,
        "Cd": 0.001,
        "m": 1.0,
}
result = simulate(t=t, parameters=parameters)

In [ ]:
labels = {
    'x':r'$x$ $[m]$',
    'v':r'$v$ $[m/s]$',
    'a':r'$a$ $[m/s^2]$',
}

#with plt.xkcd(scale=0.0, randomness=2):
fig,axes=plt.subplots(nrows=3)
for ax,key in zip(axes, ['x','v','a']):
    result.plot(y=key, ax=ax)
    ax.get_legend().set_visible(False)
    ax.set_ylabel(labels[key], rotation=90)
    ax.grid(True)
    ax.set_xticks(np.arange(0,np.max(t)+1,1));


for ax in axes[0:-1]:
    ax.set_xticklabels([])
    
axes[-1].set_xlabel('Time [s]')
plt.tight_layout()


In [ ]:
t_training = 10
result_training = result.loc[t<t_training]

model1, parameters1 = fit(result=result_training, parameters=parameters, func_features=features_simple)
model1.summary()

In [ ]:
result1_pred1 = simulate(t=result_training.index, parameters=parameters1)
fig,ax=plt.subplots()
result_training.plot(y='x', label='x measured', ax=ax)
result1_pred1.plot(y='x', style='--', label='x predicted', ax=ax)

In [ ]:
result2_pred1 = simulate(t=result.index, parameters=parameters1)

fig,ax=plt.subplots()
result.plot(y='x', label='x measured', ax=ax)

result2_pred1.plot(y='x', style='--', label='x predicted', ax=ax)

In [ ]:
model2, parameters2 = fit(result=result_training, parameters=parameters, func_features=features_drag)
model2.summary()

In [ ]:
model3, parameters3 = fit(result=result_training, parameters=parameters, func_features=features_overfit)
model3.summary()

In [ ]:
result2_pred2 = simulate(t=result.index, parameters=parameters2)
result2_pred3 = simulate(t=result.index, parameters=parameters3)

In [ ]:
set_file_name('perfect_data')


with plt.xkcd(scale=0.0, randomness=2):
    fig,ax=plt.subplots()
    
    result.plot(y='x', label='True', ax=ax)
        
    ax.set_ylabel(r'$x$ $[m]$')
    ax.set_ylim(result['x'].min(),result['x'].max())
    ax.set_xlabel('time [s]')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.50))
    snapshot()
    
    ax.fill_between(x=[0,t_training],y1=result['x'].min(),y2=result['x'].max(), alpha=0.1, zorder=-10)
    ax.annotate('<-Training data', (t_training,result['x'].min()+50))
    snapshot()
    
    label_reset()
    result2_pred1.plot(y='x', style='--', label=label(parameters1), ax=ax)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.50))
    snapshot()
    
    result2_pred2.plot(y='x', style='r--', label=label(parameters2), ax=ax)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.50))
    snapshot()
    
    result2_pred3.plot(y='x', style='m:', label=label(parameters3), ax=ax)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.50))
    snapshot()

In [ ]:
result_measure = result.copy()
np.random.seed(42)
dt = np.mean(np.diff(result_measure.index))
sigma_x = 2.0

result_measure['epsilon'] = np.random.normal(scale=sigma_x*dt, size=len(result_measure))
result_measure['x']+=result_measure['epsilon']
result_measure['v']=np.concatenate((np.diff(result_measure['x'])/dt,[0],))
result_measure['a']=np.concatenate((np.diff(result_measure['v'])/dt,[0],))

result_measure = result_measure.iloc[0:-2].copy()
result_measure_training = result_measure.loc[result_measure.index<t_training]

In [ ]:
fig,ax=plt.subplots()

result_measure.plot(y='x', label='True', ax=ax)

In [ ]:
set_file_name('measured_states')

with plt.xkcd(scale=0.0, randomness=2):
    fig,axes=plt.subplots(nrows=3)
    for ax,key in zip(axes, ['x','v','a']):
        result.plot(y=key, style='m-', label='True', ax=ax)
        ax.get_legend().set_visible(False)
        ax.set_ylabel(labels[key], rotation=90)
        ax.grid(True)
        if key!='a':
            ax.set_xticklabels([])           
        ax.set_ylim(result_measure[key].min(),result_measure[key].max())
            
    ax.set_xlabel('Time [s]')
    axes[0].legend()
    snapshot()
    
    for ax,key in zip(axes, ['x','v','a']):
        result_measure.plot(y=key, style='b-', label='Measurement', ax=ax)
        ax.get_legend().set_visible(False)
        ax.set_ylabel(labels[key], rotation=90)
        ax.grid(True)
        if key!='a':
            ax.set_xticklabels([])
        ax.set_ylim(result_measure[key].min(),result_measure[key].max())
        
    ax.set_xlabel('Time [s]')
    axes[0].legend()
    snapshot()
    
    index = result_measure['a'].idxmax()
    a_max = result_measure.loc[index]
    F_max = a_max['a']*parameters['m']
    ax.annotate(f"{int(np.round(F_max))} N!", xy=(a_max.name,F_max))
    

In [ ]:
model4, parameters4 = fit(result=result_measure_training, parameters=parameters, func_features=features_simple)
model5, parameters5 = fit(result=result_measure_training, parameters=parameters, func_features=features_drag)
model6, parameters6 = fit(result=result_measure_training, parameters=parameters, func_features=features_overfit)

display(model4.summary())
display(model5.summary())
display(model6.summary())


In [ ]:
result3_pred4 = simulate(t=result_measure.index, parameters=parameters4)
result3_pred5 = simulate(t=result_measure.index, parameters=parameters5)
result3_pred6 = simulate(t=result_measure.index, parameters=parameters6)

In [ ]:
with plt.xkcd(scale=0.0, randomness=2):
    fig,ax=plt.subplots()
    
    result_measure.plot(y='x', label='True', ax=ax)   
    label_reset()
    result3_pred4.plot(y='x', style='--', label=label(parameters4), ax=ax)
    result3_pred5.plot(y='x', style='r--', label=label(parameters5), ax=ax)
    result3_pred6.plot(y='x', style='k--', label=label(parameters6), ax=ax)
    ax.set_ylabel(r'$x$ $[m]$')
    ax.set_ylim(result['x'].min(),result['x'].max())
    ax.set_xlabel('time [s]')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.50))
    ax.fill_between(x=[0,t_training],y1=result['x'].min(),y2=result['x'].max(), alpha=0.1, zorder=-10)
    ax.annotate('<-Training data', (t_training,result['x'].min()+50))
    fig.savefig('measured_data.png', transparent=True, bbox_inches='tight', dpi=300)

In [ ]:
def wrapper(parameters):
    
    def lambda_f(x: np.ndarray, input: pd.Series) -> np.ndarray:
        return np.array(step(t=0, x=x, parameters=parameters))
    
    return lambda_f

In [ ]:
lambda_jacobian = wrapper(parameters=parameters1)

In [ ]:
from sympy import Matrix
import sympy as sp

In [ ]:
x,v,a,h = sp.symbols("x v a h")
g,m,Cd= sp.symbols("g,m,Cd")

f = Matrix([v, (-g*m+Cd*v**2)/m])
lambda_f = lambdify(f)
lambda_f

In [ ]:
f

In [ ]:


jac = sp.eye(2) + Matrix([v, (-g*m+Cd*v**2)/m]).jacobian([x, v]) * h
lambda_jacobian = lambdify(jac)
jac

In [ ]:
def wrapper(parameters):
    
    def function(x: np.ndarray, input: pd.Series) -> np.ndarray:
        v = x[1]
        return np.array(run(function=lambda_f,inputs=parameters,v=v))
    
    return function

In [ ]:
def wrapper_jacobian(parameters):
    
    def lambda_f(x: np.ndarray, input: pd.Series) -> np.ndarray:
        v = x[1]
        return np.array(run(function=lambda_jacobian,inputs=parameters,v=v, h=dt))
    
    return lambda_f



In [ ]:
parameters_guess = parameters1.copy()
#parameters_guess = parameters.copy()
parameters_guess['Cd'] = 0
parameters_guess['k'] = 0



In [ ]:
sigmas = np.array([sigma_x,10.1])
P_prd = np.diag(sigmas**2*dt)
P_prd = np.diag([0.001,0.001])

x0 = np.array([0,0])

Qd = np.array([[(15.5)**2*dt]])  # Covariance matrix of the process model
Rd = np.array([[(sigma_x)**2*dt]])  # Covariance matrix of the measurement

E = np.array([[0, 1]]).T
Cd = np.array([[1, 0]])  # Measurement!

steps = extended_kalman_filter(P_prd=P_prd, 
                               lambda_f=wrapper(parameters=parameters_guess), 
                               lambda_jacobian=wrapper_jacobian(parameters=parameters_guess), 
                               data=result_measure, 
                               Qd=Qd, 
                               Rd=Rd, 
                               E=E, 
                               Cd=Cd, 
                               state_columns=['x','v'], 
                               measurement_columns=['x'], 
                               input_columns=[], 
                               x0=x0)

Qd = np.array([[steps[-1]['P_hat'][1,1]]])  # Covariance matrix of the process model

steps = extended_kalman_filter(P_prd=steps[-1]['P_hat'], 
                               lambda_f=wrapper(parameters=parameters_guess), 
                               lambda_jacobian=wrapper_jacobian(parameters=parameters_guess), 
                               data=result_measure, 
                               Qd=Qd, 
                               Rd=Rd, 
                               E=E, 
                               Cd=Cd, 
                               state_columns=['x','v'], 
                               measurement_columns=['x'], 
                               input_columns=[], 
                               x0=x0)

In [ ]:
df_kalman = time_steps_to_df(time_steps=steps, state_columns=['x','v'])
df_kalman.rename(columns={'v1d':'a'}, inplace=True)
df_kalman_training = df_kalman.loc[df_kalman.index<t_training].copy()

In [ ]:
Ks = np.concatenate([step['K'].T for step in steps])
fig,ax=plt.subplots()
ax.plot(Ks, label=['x','v'])
ax.legend()

In [ ]:
Vs = variance(steps).T

fig,ax=plt.subplots()
ax.plot(Vs, label=['x','v'])
ax.legend()

In [ ]:
result['v']

In [ ]:
df_kalman_steps = pd.DataFrame(index=result_measure.index)
df_kalman_steps['x'] = result.iloc[0:-2]['x'].values
df_kalman_steps['x_measure'] = result_measure['x']
df_kalman_steps['x_hat'] = [step['x_hat'][0,0] for step in steps]
df_kalman_steps['x_prd'] = [0] + [step['x_prd'][0,0] for step in steps][0:-1]

set_file_name('filtering')
size=(-22,-19)

with plt.xkcd(scale=0.0, randomness=2):
    fig,ax=plt.subplots()
    fig.set_size_inches(4.0,4.0)
    #df_ = df_kalman_steps.loc[df_kalman_steps.index < 2].diff()
    df_ = df_kalman_steps.loc[(df_kalman_steps.index > 10) & (df_kalman_steps.index < 13)].diff()
    #ax.grid()
    ax.set_xlabel('Time [s]')
    ax.set_ylabel(r'$\Delta x$ $[m]$')
    
    df_.plot(y='x', label=r'True', style=':', ax=ax)
    ax.set_ylim(size)
    snapshot()
    
    df_.plot(y='x_measure', label=r'Measure', style='o--', ax=ax)
    ax.set_ylim(size)
    snapshot()
    
    df_.plot(y='x_prd', label=r'Model', style='s--', ax=ax)
    ax.set_ylim(size)
    snapshot()
    
    df_.plot(y='x_hat', label=r'Filter', style='-', ax=ax)
    ax.set_ylim(size)
    snapshot()
    
    

In [ ]:
with plt.xkcd(scale=0.0, randomness=2):
    fig,axes=plt.subplots(nrows=3)
    for ax,key in zip(axes, ['x','v','a']):
        result_measure.plot(y=key, label='Measurement', ax=ax)
        df_kalman.plot(y=key, style='-', label='Kalman filter', ax=ax)
        ax.get_legend().set_visible(False)
        ax.set_ylabel(labels[key], rotation=90)
        ax.grid(True)
        if key!='a':
            ax.set_xticklabels([])
    axes[0].legend()
    ax.set_xlabel('Time [s]')
    fig.savefig('filtered_states.png', transparent=True, bbox_inches='tight', dpi=300)

In [ ]:
model7, parameters7 = fit(result=df_kalman_training, parameters=parameters, func_features=features_simple)
model8, parameters8 = fit(result=df_kalman_training, parameters=parameters, func_features=features_drag)
model9, parameters9 = fit(result=df_kalman_training, parameters=parameters, func_features=features_overfit)

result3_pred7 = simulate(t=result_measure.index, parameters=parameters7)
result3_pred8= simulate(t=result_measure.index, parameters=parameters8)
result3_pred9 = simulate(t=result_measure.index, parameters=parameters9)

In [ ]:
with plt.xkcd(scale=0.0, randomness=2):
    fig,ax=plt.subplots()
    result.plot(y='x', label='True', ax=ax)
    
    label_reset()
    result3_pred7.plot(y='x', style='--', label=label(parameters7), ax=ax)
    result3_pred8.plot(y='x', style='r--', label=label(parameters8), ax=ax)
    result3_pred9.plot(y='x', style='k--', label=label(parameters9), ax=ax)
    ax.set_ylabel(r'$x$ $[m]$')
    ax.set_ylim(result['x'].min(),result['x'].max())
    ax.set_xlabel('time [s]')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.50))
    ax.fill_between(x=[0,t_training],y1=result['x'].min(),y2=result['x'].max(), alpha=0.1, zorder=-10)
    ax.annotate('<-Training data', (t_training,result['x'].min()+50))
    fig.savefig('filtered_data.png', transparent=True, bbox_inches='tight', dpi=300)

In [ ]:
steps2 = extended_kalman_filter(P_prd=P_prd, 
                               lambda_f=wrapper(parameters=parameters8), 
                               lambda_jacobian=wrapper_jacobian(parameters=parameters8), 
                               data=result_measure, 
                               Qd=Qd, 
                               Rd=Rd, 
                               E=E, 
                               Cd=Cd, 
                               state_columns=['x','v'], 
                               measurement_columns=['x'], 
                               input_columns=[], 
                               x0=x0)

In [ ]:
df_kalman2 = time_steps_to_df(time_steps=steps2, state_columns=['x','v'])
df_kalman2.rename(columns={'v1d':'a'}, inplace=True)
df_kalman2_training = df_kalman2.loc[df_kalman2.index<t_training].copy()

In [ ]:
model10, parameters10 = fit(result=df_kalman2_training, parameters=parameters, func_features=features_drag)
result3_pred10 = simulate(t=result_measure.index, parameters=parameters10)

In [ ]:
Qd = np.array([[(1.0)**2*dt]])  # Covariance matrix of the process model
Rd = np.array([[(sigma_x)**2*dt]])  # Covariance matrix of the measurement

steps3 = extended_kalman_filter(P_prd=P_prd, 
                               lambda_f=wrapper(parameters=parameters10), 
                               lambda_jacobian=wrapper_jacobian(parameters=parameters10), 
                               data=result_measure, 
                               Qd=Qd, 
                               Rd=Rd, 
                               E=E, 
                               Cd=Cd, 
                               state_columns=['x','v'], 
                               measurement_columns=['x'], 
                               input_columns=[], 
                               x0=x0)

df_kalman3 = time_steps_to_df(time_steps=steps3, state_columns=['x','v'])
df_kalman3.rename(columns={'v1d':'a'}, inplace=True)
df_kalman3_training = df_kalman3.loc[df_kalman3.index<t_training].copy()

model11, parameters11 = fit(result=df_kalman3_training, parameters=parameters, func_features=features_drag)
result3_pred11 = simulate(t=result_measure.index, parameters=parameters11)

In [ ]:
with plt.xkcd(scale=0.0, randomness=2):
    fig,ax=plt.subplots()
    result.plot(y='x', label='True', ax=ax)
    
    label_reset()
    ax.set_ylabel(r'$x$ $[m]$')
    ax.set_ylim(result['x'].min(),result['x'].max())
    ax.set_xlabel('time [s]')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.50))
    ax.fill_between(x=[0,t_training],y1=result['x'].min(),y2=result['x'].max(), alpha=0.1, zorder=-10)
    ax.annotate('<-Training data', (t_training,result['x'].min()+50))
    
    result3_pred10.plot(y='x', style='r--', label=label(parameters10, letter='B'), ax=ax)
    #result3_pred11.plot(y='x', style='r:', label=label(parameters11, letter='B'), ax=ax)
    fig.savefig('refiltered_data.png', transparent=True, bbox_inches='tight', dpi=300)

In [ ]:
df_kalman_steps3[['x','x_hat']]

In [ ]:
result_measure